In [ ]:
#!pip install openpyxl
#anndata needs to be 0.8.0
#!pip install -U kaleido
from glob import glob
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import anndata
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import tarfile
import seaborn as sns
import torch

from collections import OrderedDict
import pickle
#import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import os

In [ ]:
import plotly

In [ ]:
tcell_ann_concat = anndata.read_h5ad('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/tcell_ann_concat_Tcell_only_Migration.h5ad')

In [ ]:
list(tcell_ann_concat.obs.columns)

In [ ]:
### Remove small clusters
# Get the value counts
cluster_counts = tcell_ann_concat.obs['leiden_0.5'].value_counts()

# Filter out clusters with counts lower than 200
filtered_clusters = cluster_counts[cluster_counts >= 200]

# Get the names of the clusters to keep
clusters_to_keep = filtered_clusters.index

# Filter the AnnData object to keep only the cells in those clusters
filtered_tcell_ann_concat = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'].isin(clusters_to_keep)]

# Optionally, view the filtered counts
filtered_counts = filtered_tcell_ann_concat.obs['leiden_0.5'].value_counts()
filtered_counts

tcell_ann_concat = filtered_tcell_ann_concat

new_cluster_names = ['CD8+ Effector T cells','CD4+ Effector T cells','CD8+ Tissue Resident Memory T cells', "CD4+ Central Memory T cells","CD8+ Cytotoxic Unconventional T cells",'Low CD3',"Proliferating CD8+ T cells","CD4+ Regulatory T cells",'CD8+ Homeostatic Unconventional T cells'
]
tcell_ann_concat.rename_categories('leiden_0.5', new_cluster_names)

In [ ]:
tcell_ann_concat_Mobile = tcell_ann_concat[tcell_ann_concat.obs['Migration_all'] =="Mobile"]

In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile, color=['leiden_0.5','LPorIE'])

In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'].isin(['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'])], color=['leiden_0.5','LPorIE'])

In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'].isin(['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'])], color='IL7R')

In [ ]:
tcell_ann_concat_Mobile.obs['leiden_LPorIE'] = tcell_ann_concat_Mobile.obs['leiden_0.5'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['LPorIE'].astype(str)

In [ ]:
tcell_ann_concat_Mobile_conv = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'].isin(['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'] )]

In [ ]:
tcell_ann_concat_IE = tcell_ann_concat[tcell_ann_concat.obs['Migration_all'] =="IE_Stationary"]

In [ ]:
# Count occurrences of each cell type in 'leiden_0.5'
cell_type_counts = tcell_ann_concat_IE.obs['leiden_0.5'].value_counts()

# Display the counts
print(cell_type_counts)


In [ ]:
tcell_ann_concat_LP = tcell_ann_concat[tcell_ann_concat.obs['Migration_all'] =="LP_Stationary"]
# Count occurrences of each cell type in 'leiden_0.5'
cell_type_counts = tcell_ann_concat_LP.obs['leiden_0.5'].value_counts()

# Display the counts
print(cell_type_counts)

In [ ]:
tcell_ann_concat.obs['LPorIE']

In [ ]:
tcell_ann_concat_LP = tcell_ann_concat[tcell_ann_concat.obs['LPorIE'] =="Lamina Propria Cells"]
# Count occurrences of each cell type in 'leiden_0.5'
cell_type_counts = tcell_ann_concat_LP.obs['leiden_0.5'].value_counts()

# Display the counts
print(cell_type_counts)

In [ ]:
tcell_ann_concat_IE = tcell_ann_concat[tcell_ann_concat.obs['LPorIE'] =='Intraepithelial Cells']
# Count occurrences of each cell type in 'leiden_0.5'
cell_type_counts = tcell_ann_concat_IE.obs['leiden_0.5'].value_counts()

# Display the counts
print(cell_type_counts)

In [ ]:
tcell_ann_concat.obs['SampleID_LPorIE'] = tcell_ann_concat.obs['SampleID'].astype(str)+'_'+tcell_ann_concat.obs['LPorIE'].astype(str)

In [ ]:
sc.pl.umap(tcell_ann_concat, color='SampleID_LPorIE')

In [ ]:
sc.pl.umap(tcell_ann_concat, color=['leiden_0.5'])

In [ ]:

plt.rcParams.update({'font.size': 16}) 

sc.pl.umap(
    tcell_ann_concat_Mobile_conv,
    color=['ITGAE','ITGA1','GZMA','GZMB','CTLA4','PDCD1','IL7R',"GVHD_Grade_1"],
    show=False,  # Do not show the plot yet
    frameon=False,
    size=10
)

# Save the figure at 300 DPI
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/mobile_conv_leiden_il7_grade_umap.png", dpi=300)
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/mpbile_conv_leiden_il7_grade_umap.pdf", dpi=300)
# Optionally, show the plot after saving
plt.show()

In [ ]:
tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v_Grade'] = tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['PatientID'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['Dates_from_Transplantation'].astype(str)

In [ ]:
import plotly.graph_objects as go
from anndata import AnnData
from scipy.stats import chi2_contingency
### Remove them

link_colors = [
    'rgba(31, 119, 180, 0.3)',  # Blue for 'Infiltrating activated cytotoxic T-cells'
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(44, 160, 44, 0.3)',   # Green for 'Resident CD8+ T-cells'
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(227, 119, 194, 0.3)', # Pink for 'Proliferating CD8+ T-cells'
    'rgba(227, 119, 194, 0.3)', # Pink
    'rgba(227, 119, 194, 0.3)'  # Pink
]

for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_0.5', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    print(normalized_pivot_table)
    
    df= pd.DataFrame()
    df_before =pd.DataFrame()
    total_migrate_count = 0
    total_lp_count = 0
    accumulated_adata_migrate = []
    accumulated_adata_lp_count = []
    for t_cell_states in tcell_ann_concat_Mobile_conv.obs['leiden_0.5'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_0.5'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_lp.obs['cc_aa_alignment_same_v_Grade'])]
        adata_lp_count= adata_lp[adata_lp.obs['cc_aa_alignment_same_v_Grade'].isin(adata_migrate.obs['cc_aa_alignment_same_v_Grade'])]
        total_migrate_count += len(adata_migrate)
        total_lp_count += len(adata_lp_count)
        
        print(len(adata_migrate))
        print(len(adata_lp_count))
        # Concatenate the new data to the accumulated data
        accumulated_adata_migrate = accumulated_adata_migrate+list(adata_migrate.obs.index)
        accumulated_adata_lp_count = accumulated_adata_lp_count+list(adata_lp_count.obs.index)
        
        adata_migrate.obs['leiden_0.5'] = pd.Categorical(adata_migrate.obs['leiden_0.5'], categories=['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'])
        df[t_cell_states] = adata_migrate.obs['leiden_0.5'].value_counts(normalize=True) * normalized_pivot_table['Lamina Propria Cells'][t_cell_states]
        #print(adata_migrate.obs['leiden_new_cluster_2'].unique())

    
    df.fillna(0, inplace=True)

    if (df == 0).any().any():
        print("The DataFrame contains 0.")
    else:
        print("The DataFrame does not contain any 0.")
    
    print('df')
    print(df)
    print('sum of counts')
    print("Total adata_migrate count:", total_migrate_count)
    print("Total adata_lp_count count:", total_lp_count)
    print('sum of unique cells')
    print("Total adata_migrate count:", len(list(set(accumulated_adata_migrate))))
    print("Total adata_lp_count count:", len(list(set(accumulated_adata_lp_count))))


    

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = ["CD8+_Effector", "CD8+_Tissue_Resident_Memory", "Proliferating_CD8+","CD8+_Effector", "CD8+_Tissue_Resident_Memory", "Proliferating_CD8+"],
          color = ['#1f77b4','#1b7837','#e377c2','#1f77b4','#1b7837','#e377c2']
        ),
        link = dict(
          source = [0, 0, 0, 1, 1, 1,2,2,2], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [3, 4, 5, 3, 4, 5,3,4,5],
          value = [df['CD8+ Effector T cells']['CD8+ Effector T cells'], df['CD8+ Effector T cells']['CD8+ Tissue Resident Memory T cells'] , df['CD8+ Effector T cells']['Proliferating CD8+ T cells'],  df['CD8+ Tissue Resident Memory T cells']['CD8+ Effector T cells'] , df['CD8+ Tissue Resident Memory T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Tissue Resident Memory T cells']['Proliferating CD8+ T cells'],df['Proliferating CD8+ T cells']['CD8+ Effector T cells'], df['Proliferating CD8+ T cells']['CD8+ Tissue Resident Memory T cells'],df['Proliferating CD8+ T cells']['Proliferating CD8+ T cells']],
            color = link_colors,
      ))])
    
    fig.update_layout(title_text=i,
                      font=dict(size = 15, color = 'black'))
    fig.show()
        # Save as PNG
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram_3.png", scale=4)
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram_3.pdf", scale=4)

 # Access the .obs 
from scipy.stats import fisher_exact
for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_0.5', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    normalized_pivot_table = normalized_pivot_table*100
    print(normalized_pivot_table)
    try:
        # Perform chi-square test
        chi2_stat, p_val, dof, ex = chi2_contingency(normalized_pivot_table)
    
        # Check if there are any zeroes in the expected frequencies
        if np.any(ex == 0):
            print("Skipping this chi-square test due to zero expected frequencies.")
        else:
            # Output the results if the test is valid
            print(f"Chi-square statistic: {chi2_stat}")
            print(f"P-value: {p_val}")
            print(f"Degrees of freedom: {dof}")
            print(f"Expected frequencies:\n{ex}")
    
    except ValueError as e:
        print(f"Skipping this test due to error: {e}")

In [ ]:

### TO do  review change resi code
import plotly.graph_objects as go
link_colors = [
    'rgba(31, 119, 180, 0.3)',  # Blue for 'Infiltrating activated cytotoxic T-cells'
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(44, 160, 44, 0.3)',   # Green for 'Resident CD8+ T-cells'
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(227, 119, 194, 0.3)', # Pink for 'Proliferating CD8+ T-cells'
    'rgba(227, 119, 194, 0.3)', # Pink
    'rgba(227, 119, 194, 0.3)'  # Pink
]

Change_State_Resi = []
activated_state = pd.DataFrame()
proliferating_state = pd.DataFrame()
resident_state = pd.DataFrame()
for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    df= pd.DataFrame()
    df_before =pd.DataFrame()
    for t_cell_states in tcell_ann_concat_Mobile_conv.obs['leiden_0.5'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_0.5'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_lp.obs['cc_aa_alignment_same_v_Grade'])]
        print(len(adata_migrate))
        
        adata_migrate.obs['leiden_0.5'] = pd.Categorical(adata_migrate.obs['leiden_0.5'], categories=['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'])
        adata_ie_resident = adata_ie[adata_ie.obs['leiden_0.5'] == 'CD8+ Tissue Resident Memory T cells']
        adata_ie_resident_migrated_lp = adata_lp[adata_lp.obs['cc_aa_alignment_same_v_Grade'].isin(adata_ie_resident.obs['cc_aa_alignment_same_v_Grade'])]
        adata_ie_resident_migrated_lp_not_resident = adata_ie_resident_migrated_lp[adata_ie_resident_migrated_lp.obs['leiden_0.5'] != 'Resident CD8+ T-cells']
        adata_ie_resident_change = adata_ie_resident[adata_ie_resident.obs['cc_aa_alignment_same_v_Grade'].isin(adata_ie_resident_migrated_lp_not_resident.obs['cc_aa_alignment_same_v_Grade'])]

        print(adata_ie_resident_change.obs.index)
        Change_State_Resi.append(list(adata_ie_resident_change.obs.index))
        df[t_cell_states] = adata_migrate.obs['leiden_0.5'].value_counts(normalize=True)
        print(adata_migrate.obs['leiden_0.5'].unique())

    print(df)
    activated_state[i] = df['CD8+ Effector T cells']
    proliferating_state[i] = df['Proliferating CD8+ T cells']
    resident_state[i] = df['CD8+ Tissue Resident Memory T cells']
    df.fillna(0, inplace=True)

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = ["LP_CD8+_Effector", "LP_CD8+_Tissue_Resident_Memory", "LP_Proliferating_CD8+","IE_CD8+_Effector", "IE_CD8+_Tissue_Resident_Memory", "IE_Proliferating_CD8+"],
          color = ['#1f77b4','#1b7837','#e377c2','#1f77b4','#1b7837','#e377c2']
        ),
        link = dict(
          source = [0, 0, 0, 1, 1, 1,2,2,2], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [3, 4, 5, 3, 4, 5,3,4,5],
          value = [df['CD8+ Effector T cells']['CD8+ Effector T cells'], df['CD8+ Effector T cells']['CD8+ Tissue Resident Memory T cells'] , df['CD8+ Effector T cells']['Proliferating CD8+ T cells'],  df['CD8+ Tissue Resident Memory T cells']['CD8+ Effector T cells'] , df['CD8+ Tissue Resident Memory T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Tissue Resident Memory T cells']['Proliferating CD8+ T cells'],df['Proliferating CD8+ T cells']['CD8+ Effector T cells'], df['Proliferating CD8+ T cells']['CD8+ Tissue Resident Memory T cells'],df['Proliferating CD8+ T cells']['Proliferating CD8+ T cells']],
            color = link_colors,
      ))])
    
    fig.update_layout(title_text=i, font_size=10)
    fig.show()

In [ ]:
#sc.settings.figdir = '/burg/iicd_new/users/ls3456/GVHD/Figures/Trajectory_figures/'
plt.figure(figsize=(15, 6))
# Now the plot will be saved in the specified directory
sc.pl.umap(tcell_ann_concat_Mobile_conv[np.random.permutation(len(tcell_ann_concat_Mobile_conv))],
    color=['leiden_0.5'],
    show=False,  # Do not show the plot yet,
    frameon=False,
    size=80,
    legend_loc= 'right margin',
    legend_fontsize = 12)

# Save the figure at 300 DPI
plt.savefig("//Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Mobile_conv_umap.png", dpi=300,bbox_inches='tight')
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Mobile_conv_umap.pdf", dpi=300,bbox_inches='tight')

In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile, color='leiden_LPorIE')
# plot clones on decipher trajectory

tcell_ann_concat_Mobile.obs['Change_Resi'] = 'No'

tcell_ann_concat_Mobile.obs.loc[tcell_ann_concat_Mobile.obs.index.isin(Change_State_Resi[1]),'Change_Resi'] = "Yes"

sc.pl.umap(tcell_ann_concat_Mobile, color='Change_Resi')
# plot clones on decipher trajectory

sc.pl.umap(tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'] == 'CD8+ Tissue Resident Memory T cells'], color='Change_Resi')
# plot clones on decipher trajectory

Change_Resi = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'] == 'CD8+ Tissue Resident Memory T cells']

sc.pl.umap(Change_Resi, color=['leiden_0.5','Change_Resi','IL7R','GVHD_Grade_1'])

In [ ]:
list(tcell_ann_concat_Mobile.obs['leiden_0.5'].unique())

In [ ]:
# Step 1: Add IE and LP colors to the provided color map for CD8 T-cell types
cd8_colors = {
    'CD8+ Effector T cells': '#1f77b4',
    'Proliferating CD8+ T cells': '#e377c2',
    'CD8+ Tissue Resident Memory T cells': '#2ca02c',
    #'CD8+ Cytotoxic Unconventional T cells': '#9467bd',
    #'CD8+ Homeostatic Unconventional T cells': '#bcbd22',
    'Intraepithelial Cells': 'black',  # Custom color for IE
    'Lamina Propria Cells': 'red'    # Custom color for LP
}

tcell_ann_concat_Mobile.obs['cc_aa_alignment_same_v_Grade'] = tcell_ann_concat_Mobile.obs['cc_aa_alignment_same_v'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['PatientID'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['Dates_from_Transplantation'].astype(str)
Change_Resi.obs['cc_aa_alignment_same_v_Grade'] = Change_Resi.obs['cc_aa_alignment_same_v'].astype(str)+'_'+Change_Resi.obs['GVHD_Grade_1'].astype(str)+'_'+Change_Resi.obs['PatientID'].astype(str)+'_'+Change_Resi.obs['Tissue_Location'].astype(str)+'_'+Change_Resi.obs['Dates_from_Transplantation'].astype(str)

Change_Resi_real = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['Change_Resi']== 'Yes']
# Step 1: Filter for Proliferating CD8+ T-cells
prolif_cd8_data = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v'].isin(Change_Resi_real.obs['cc_aa_alignment_same_v'])]

clone_counts = tcell_ann_concat_Mobile_conv.obs.groupby('cc_aa_alignment_same_v_Grade').size()

# Step 3: Get the top 10 clones by counts
top_10_clones = clone_counts.nlargest(30).index

# Step 4: Loop through each of the top clones and create individual plots
for i, value in enumerate(top_10_clones):
    # Create a new figure for each plot
    plt.figure(figsize=(8, 6))

    # Plot all cells colored by their corresponding cell types from cd8_colors
    
    for cell_type, color in cd8_colors.items():
        cell_data = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'] == cell_type]
        plt.scatter(cell_data.obsm['X_umap'][:, 0], cell_data.obsm['X_umap'][:, 1],
                    c=color, s=5, alpha=0.9, label=cell_type, edgecolor = "none", marker='o')

    # Subset the data for the current clone
    clone_data = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v_Grade'] == value]
    print(list(clone_data.obs['leiden_0.5'].unique()))

    # Map 'LPorIE' values to colors using the cd8_colors map
    clone_colors = clone_data.obs['LPorIE'].map(cd8_colors)

    # Plot the current clone with a color map based on LPorIE (Intraepithelial or Lamina Propria)
    scatter = plt.scatter(clone_data.obsm['X_umap'][:, 0], clone_data.obsm['X_umap'][:, 1],
                          c=clone_colors, s=50, alpha=1, label=f'Clone {value}', edgecolor = "none", marker='o')

    # Adding colorbar, labels, and title
    #plt.title(f'Clone {value} ')
    plt.gca().set_axis_off()
    
    # plt.xlabel('Decipher Component 1')
    # plt.ylabel('Decipher Component 2')

    # Add legend
    #plt.legend(loc='upper right', bbox_to_anchor=(2, 1))
    print(i)
    print(value)

    # Show the plot for the current clone
    
            # Save figure with 300 DPI
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.svg', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Create a new figure just for the legend
plt.figure(figsize=(3, 1.5))

# Create legend handles with circular markers
from matplotlib.lines import Line2D

ie_marker = Line2D([0], [0], marker='o', color='w', markerfacecolor='black', 
                  markersize=15, label='Intraepithelial Cells')
lp_marker = Line2D([0], [0], marker='o', color='w', markerfacecolor='red', 
                  markersize=15, label='Lamina Propria Cells')

# Add the legend
plt.legend(handles=[ie_marker, lp_marker], loc='center', frameon=False)

# Remove the axes
plt.gca().set_axis_off()

# Save the legend as a separate file
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/legend_IE_LP.png', dpi=300, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/legend_IE_LP.pdf', dpi=300, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/legend_IE_LP.svg', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Step 1: Add IE and LP colors to the provided color map for CD8 T-cell types
cd8_colors = {
    'CD8+ Effector T cells': '#1f77b4',
    'Proliferating CD8+ T cells': '#e377c2',
    'CD8+ Tissue Resident Memory T cells': '#2ca02c',
    'CD8+ Cytotoxic Unconventional T cells': '#9467bd',
    'CD8+ Homeostatic Unconventional T cells': '#bcbd22',
    'Intraepithelial Cells': 'black',  # Custom color for IE
    'Lamina Propria Cells': 'red'    # Custom color for LP
}

tcell_ann_concat_Mobile.obs['cc_aa_alignment_same_v_Grade'] = tcell_ann_concat_Mobile.obs['cc_aa_alignment_same_v'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['PatientID'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat_Mobile.obs['Dates_from_Transplantation'].astype(str)
Change_Resi.obs['cc_aa_alignment_same_v_Grade'] = Change_Resi.obs['cc_aa_alignment_same_v'].astype(str)+'_'+Change_Resi.obs['GVHD_Grade_1'].astype(str)+'_'+Change_Resi.obs['PatientID'].astype(str)+'_'+Change_Resi.obs['Tissue_Location'].astype(str)+'_'+Change_Resi.obs['Dates_from_Transplantation'].astype(str)

Change_Resi_real = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['Change_Resi']== 'Yes']
# Step 1: Filter for Proliferating CD8+ T-cells
prolif_cd8_data = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v'].isin(Change_Resi_real.obs['cc_aa_alignment_same_v'])]

clone_counts = Change_Resi_real.obs.groupby('cc_aa_alignment_same_v_Grade').size()

# Step 3: Get the top 10 clones by counts
top_10_clones = clone_counts.nlargest(30).index

# Step 4: Loop through each of the top clones and create individual plots
for i, value in enumerate(top_10_clones):
    # Create a new figure for each plot
    plt.figure(figsize=(8, 6))

    # Plot all cells colored by their corresponding cell types from cd8_colors
    
    for cell_type, color in cd8_colors.items():
        cell_data = tcell_ann_concat[tcell_ann_concat.obs['leiden_0.5'] == cell_type]
        plt.scatter(cell_data.obsm['X_umap'][:, 0], cell_data.obsm['X_umap'][:, 1],
                    c=color, s=5, alpha=0.9, label=cell_type, edgecolor = "none", marker='o')

    # Subset the data for the current clone
    clone_data = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v_Grade'] == value]
    print(list(clone_data.obs['leiden_0.5'].unique()))

    # Map 'LPorIE' values to colors using the cd8_colors map
    clone_colors = clone_data.obs['LPorIE'].map(cd8_colors)

    # Plot the current clone with a color map based on LPorIE (Intraepithelial or Lamina Propria)
    scatter = plt.scatter(clone_data.obsm['X_umap'][:, 0], clone_data.obsm['X_umap'][:, 1],
                          c=clone_colors, s=50, alpha=1, label=f'Clone {value}', edgecolor = "none", marker='o')

    # Adding colorbar, labels, and title
    #plt.title(f'Clone {value} ')
    plt.gca().set_axis_off()
    
    # plt.xlabel('Decipher Component 1')
    # plt.ylabel('Decipher Component 2')

    # Add legend
    #plt.legend(loc='upper right', bbox_to_anchor=(2, 1))

    # Show the plot for the current clone
    print(i)
    print(value)
            # Save figure with 300 DPI
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_{value}_umap_300dpi.svg', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile, color=['leiden_0.5'])

In [ ]:
#Change_Resi[Change_Resi.obs["Change_Resi"]== "Yes"].obs['GVHD_Grade_1']

In [ ]:
#Change_Resi.write('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/Change_Resi.h5ad')

In [ ]:
#sc.tl.rank_genes_groups(Change_Resi, groupby='Change_Resi', use_raw=False, method='wilcoxon')
#degs_df = sc.get.rank_genes_groups_df(Change_Resi, group='Yes')

In [ ]:
#degs_df.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/changeofResident_cluster.csv')

In [ ]:
#list(degs_df.head(200)['names'])

degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]


crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(3,3), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=3)

            

         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Coverted Resident T cells vs Orginal Resident', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/Trajectory_figures/converted_resident_vs_original_resident.pdf', dpi=300)
plt.savefig('/burg/iicd/users/ls3456/GVHD/Figures/Trajectory_figures/converted_resident_vs_original_resident.png', dpi=300)
plt.show()

In [ ]:
Resi = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'] == 'CD8+ Tissue Resident Memory T cells']

In [ ]:
#!pip3 install leidenalg

In [ ]:
# Compute the nearest neighbors
sc.pp.neighbors(Resi,use_rep="X_scVI")

# Run Leiden clustering
#sc.tl.leiden(Resi, resolution=0.09)  # You can adjust the resolution for more/less clusters
sc.tl.leiden(Resi, resolution=0.09)  # You can adjust the resolution for more/less clusters

In [ ]:
# Plot the UMAP with reclustered cells
sc.pl.umap(Resi, color=['leiden','IL7R'])

#sc.settings.figdir = '/burg/iicd_new/users/ls3456/GVHD/Figures/Trajectory_figures/'

# Now the plot will be saved in the specified directory
sc.pl.umap(Resi,
    color=['leiden','IL7R'],
    show=False,  # Do not show the plot yet,
    frameon=False,
    size=80
)

# Save the figure at 300 DPI
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Resi_umap.png", dpi=300)
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Resi_umap.pdf", dpi=300)

In [ ]:
#Resi.write('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/Resi.h5ad')

In [ ]:
sc.tl.rank_genes_groups(Resi, groupby='leiden', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(Resi, group = None)

In [ ]:
degs_df.to_csv('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/anndata_file/tworesident_cluster.csv')

In [ ]:
degs_df

In [ ]:
#sc.pl.umap(Resi, color = degs_df[degs_df['group'] =="1"].head(100)['names'])

In [ ]:
#sc.pl.umap(Resi, color = degs_df[degs_df['group'] =="0"].head(100)['names'])

In [ ]:
sc.pl.umap(Resi, color = ['TMIGD2','CD248','ITGA1','ITGAE','CD101'])

In [ ]:
sc.pl.umap(Resi, color = ['TIGIT','HAVCR2','LAG3'])

In [ ]:
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]

crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(3,3), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=2)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('1 vs 0', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]

crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']
marker= ['ITGAE','CD160','IL7R','CTLA4','PDCD1','GZMA','GZMB','ENTPD1','TOX2','ZNF683']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(3,3), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=6)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Transitioning Resident vs \nTissue Resident Memory', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)

plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Transi_Resi_vs_Tissue_Resident_volcano_plot.png', format='png', dpi=300, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Transi_Resi_vs_Tissue_Resident_volcano_plot.pdf', format='pdf', dpi=300, bbox_inches='tight')

plt.show()



In [ ]:
#Change_Resi[Change_Resi.obs['Change_Resi']=='Yes']

In [ ]:
#Resi[Resi.obs['leiden']=='1']

In [ ]:
# Define the order you want
desired_order = ['ND','No', 'Mild', 'Severe']  # Adjust based on your data

# Convert to categorical with specified order
Resi.obs["GVHD_Grade_1"] = pd.Categorical(
    Resi.obs["GVHD_Grade_1"],
    categories=desired_order,
    ordered=True
)
Resi.uns["GVHD_Grade_1_colors"] = ['#09BB8C','#09BB8C','#005A8F', '#B85000']

In [ ]:
#len(np.intersect1d(Change_Resi[Change_Resi.obs['Change_Resi']=='Yes'].obs.index, Resi[Resi.obs['leiden']=='1'].obs.index))

In [ ]:

import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'viridis'  # Or your preferred colormap
mpl.rcParams.update({'font.size': 8,
                    'svg.fonttype': 'none'})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['xtick.labelsize'] = 8  # Colorbar tick label size 
mpl.rcParams['ytick.labelsize'] = 8  # Colorbar tick label size


fig = sc.pl.umap(
    Resi,
    color=['ITGAE','GZMA','GZMB','CTLA4','PDCD1','IL7R','ZNF683',"GVHD_Grade_1"],
    show=False,  # Do not show the plot yet
    frameon=False,
    #size=80,
    vmin=0,  # Set the minimum value for the IL7R color scale
    vmax=2.75,
    #legend_loc=None,  # Remove legend
    title=None,       # Remove title
    return_fig = True,
    s = 10
)
# Access the current Axes and update the title font size

fig.set_size_inches(6.5, 2.2)

plt.tight_layout()

# Save the figure at 300 DPI
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_il7_grade_umap.png", dpi=300, transparent=True)
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_il7_grade_umap.pdf", dpi=300, transparent=True)
fig.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_il7_grade_umap.svg", dpi=300, transparent=True)
# Optionally, show the plot after saving
plt.show()

In [ ]:
sc.tl.rank_genes_groups(Resi, groupby='GVHD_Grade_1', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(Resi, group = 'Severe')

In [ ]:
degs_df.head(100)

In [ ]:
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]

crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(3,3), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=2)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Severe vs Rest', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
degs_df = degs_df.loc[np.abs(degs_df.logfoldchanges) < 10]

crit_pval = 0.001
crit_l2fc = 1
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

marker_intersect = sig_degs_df['names']
marker= ['ITGAE','CD160','IL7R','CTLA4','PDCD1','GZMA','GZMB','ENTPD1','TOX2','ZNF683']

crit_pval = 0.05
crit_l2fc = 0.5
nonsig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) <= crit_l2fc) & 
                           (degs_df['pvals_adj'] >= crit_pval))]
sig_degs_df = degs_df.loc[((np.abs(degs_df['logfoldchanges']) > crit_l2fc) & 
                           (degs_df['pvals_adj'] < crit_pval))]

fig, ax = plt.subplots(figsize=(3,3), dpi=300)

ax.scatter(degs_df['logfoldchanges'], 
            degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), c='gray',
            s=3)

ax.scatter(nonsig_degs_df['logfoldchanges'], 
            nonsig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            s=3)

ax.scatter(sig_degs_df['logfoldchanges'], 
            sig_degs_df['pvals_adj'].apply(lambda x: -np.log10(x)), 
            c='g',
            s=3)

ax.scatter(sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['logfoldchanges'],
            sig_degs_df.loc[degs_df['names'].isin(marker_intersect)]['pvals_adj'].apply(lambda x: -np.log10(x)),
            c='r',
            s=3)

ax.axvline(x=crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axvline(x=-crit_l2fc, c='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=-np.log10(crit_pval), c='gray', linestyle='--', linewidth=0.5)

oxphos_lfcs = sig_degs_df.loc[degs_df['names'].isin(marker)]['logfoldchanges']
oxphos_pvals = sig_degs_df.loc[degs_df['names'].isin(marker)]['pvals_adj']
oxphos_genes = sig_degs_df.loc[degs_df['names'].isin(marker)]['names']


#ax.set_xlim(-2, 2)  # Adjust these limits as needed
#ax.set_ylim(-1, 30)  # Adjust these limits as needed


for lfc, pval, gene in zip(oxphos_lfcs, oxphos_pvals, oxphos_genes):
    ax.text(lfc+0.1, -np.log10(pval)+0.5, gene, fontsize=6)
         
ax.set_xlabel('Log2FC', fontsize=12)  # Adjust fontsize as needed
ax.set_ylabel('-log10(adj.pvalue)', fontsize=12)  # Adjust fontsize as needed
ax.set_title('Severe Vs Rest', fontsize=12)
ax.spines[['right', 'top']].set_visible(False)

plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Transi_Resi_vs_Tissue_Resident_volcano_plot.png', format='png', dpi=300, bbox_inches='tight')
plt.savefig('/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Transi_Resi_vs_Tissue_Resident_volcano_plot.pdf', format='pdf', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
def plot_marker_heatmap(adata, gene_list, groupby='Severe', figsize=(2, 1.5)):
    """
    Plot a heatmap of z-scored expression values for marker genes.
    
    Parameters:
    -----------
    adata : AnnData
        The annotated data matrix.
    gene_list : list
        List of gene names to include in the heatmap.
    groupby : str
        Column in adata.obs to group cells by.
    figsize : tuple
        Figure size (width, height).
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        The figure object containing the heatmap.
    """
    # Check if all genes are in the dataset
    missing_genes = [gene for gene in gene_list if gene not in adata.var_names]
    if missing_genes:
        print(f"Warning: The following genes are not in the dataset: {missing_genes}")
        gene_list = [gene for gene in gene_list if gene in adata.var_names]
    
    if len(gene_list) == 0:
        print("No valid genes to plot!")
        return None
    
    # Create a copy of the AnnData object with only the genes of interest
    adata_subset = adata[:, gene_list].copy()
    
    # Z-score the data (scale each gene)
    sc.pp.scale(adata_subset, max_value=10)
    
    # Get average expression by group
    if groupby in adata_subset.obs.columns:
        # Clear current figure if any
        plt.clf()
        
        # First, compute dendrogram
        sc.tl.dendrogram(adata_subset, groupby=groupby)
        
        # Now create the heatmap
        sc.pl.heatmap(adata_subset, 
                      var_names=gene_list, 
                      groupby=groupby,
                      dendrogram=True,
                      swap_axes=True,
                      show_gene_labels=True,
                      cmap='viridis',
                      vmin=-2,
                      vmax=2,
                      figsize=figsize,
                      show=False)
        
        # Get the current figure
        fig = plt.gcf()
        plt.tight_layout()
        
        return fig
    else:
        print(f"Error: '{groupby}' column not found in adata.obs")
        return None

marker= ['ITGAE','CD160','IL7R','PDCD1','ZNF683','CTLA4','LAG3','GZMA','GZMB','ENTPD1','TOX2']
fig = plot_marker_heatmap(Resi, marker, groupby='GVHD_Grade_1', figsize = (6,2.5))

# Save figure with 300 DPI
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Resi_gene_deg.png', dpi=300, bbox_inches='tight')
fig.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/Resi_gene_deg.pdf', dpi=300, bbox_inches='tight')

In [ ]:
#tcell_ann_concat_Mobile.obs['leiden_new_cluster'] =tcell_ann_concat_Mobile.obs['leiden_0.5']
#tcell_ann_concat_Mobile.obs['leiden_new_cluster'] = tcell_ann_concat_Mobile.obs['leiden_new_cluster'].cat.add_categories(['Transitioning Resident'])
#tcell_ann_concat_Mobile.obs.loc[tcell_ann_concat_Mobile.obs.index.isin(Change_Resi[Change_Resi.obs['Change_Resi']=='Yes'].obs.index),'leiden_new_cluster'] = 'CTLA4+GZMA+Resident'
#tcell_ann_concat_Mobile.obs['leiden_new_cluster'] = tcell_ann_concat_Mobile.obs['leiden_new_cluster'].cat.add_categories(['IL7R+Resident'])
#tcell_ann_concat_Mobile.obs.loc[tcell_ann_concat_Mobile.obs.index.isin(Change_Resi[Change_Resi.obs['Change_Resi']=='No'].obs.index),'leiden_new_cluster'] = 'IL7R+Resident'

tcell_ann_concat_Mobile.obs['leiden_new_cluster_2'] =tcell_ann_concat_Mobile.obs['leiden_0.5']
tcell_ann_concat_Mobile.obs['leiden_new_cluster_2'] = tcell_ann_concat_Mobile.obs['leiden_new_cluster_2'].cat.add_categories(['CD8+ Transitioning Resident T cells'])
tcell_ann_concat_Mobile.obs.loc[tcell_ann_concat_Mobile.obs.index.isin(Resi[Resi.obs['leiden']=='1'].obs.index),'leiden_new_cluster_2'] = 'CD8+ Transitioning Resident T cells'



tcell_ann_concat_Mobile_conv = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'].isin(['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells'] )]
tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v_Grade'] = tcell_ann_concat_Mobile_conv.obs['cc_aa_alignment_same_v'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['PatientID'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['Tissue_Location'].astype(str)+'_'+tcell_ann_concat_Mobile_conv.obs['Dates_from_Transplantation'].astype(str)

In [ ]:
#tcell_ann_concat.obs['leiden_new_cluster'] =tcell_ann_concat.obs['leiden_0.5']
#tcell_ann_concat.obs['leiden_new_cluster'] = tcell_ann_concat.obs['leiden_new_cluster'].cat.add_categories(['CTLA4+GZMA+Resident'])
#tcell_ann_concat.obs.loc[tcell_ann_concat.obs.index.isin(Change_Resi[Change_Resi.obs['Change_Resi']=='Yes'].obs.index),'leiden_new_cluster'] = 'CTLA4+GZMA+Resident'
#tcell_ann_concat.obs['leiden_new_cluster'] = tcell_ann_concat.obs['leiden_new_cluster'].cat.add_categories(['IL7R+Resident'])
#tcell_ann_concat.obs.loc[tcell_ann_concat.obs.index.isin(Change_Resi[Change_Resi.obs['Change_Resi']=='No'].obs.index),'leiden_new_cluster'] = 'IL7R+Resident'

tcell_ann_concat.obs['leiden_new_cluster_2'] =tcell_ann_concat.obs['leiden_0.5']
tcell_ann_concat.obs['leiden_new_cluster_2'] = tcell_ann_concat.obs['leiden_new_cluster_2'].cat.add_categories(['CD8+ Transitioning Resident T cells'])
tcell_ann_concat.obs.loc[tcell_ann_concat.obs.index.isin(Resi[Resi.obs['leiden']=='1'].obs.index),'leiden_new_cluster_2'] = 'CD8+ Transitioning Resident T cells'
#tcell_ann_concat.obs['leiden_new_cluster_2'] = tcell_ann_concat.obs['leiden_new_cluster_2'].cat.add_categories(['IL7R+Resident'])
#tcell_ann_concat.obs.loc[tcell_ann_concat.obs.index.isin(Resi[Resi.obs['leiden']=='0'].obs.index),'leiden_new_cluster_2'] = 'IL7R+Resident'

In [ ]:
tcell_ann_concat.obs['leiden_new_cluster_2']

In [ ]:
sc.tl.rank_genes_groups(tcell_ann_concat, groupby='leiden_new_cluster_2', use_raw=False, method='wilcoxon')

In [ ]:
degs_df = sc.get.rank_genes_groups_df(tcell_ann_concat, group = None)

In [ ]:
#degs_df.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/spatial_sig.csv')

In [ ]:
# Plot the UMAP with reclustered cells
sc.pl.umap(tcell_ann_concat_Mobile_conv, color=['leiden_new_cluster_2','SampleID_Grade' ])

In [ ]:
existing_colors = tcell_ann_concat_Mobile_conv.uns['leiden_0.5_colors']

In [ ]:
existing_colors

In [ ]:
tcell_ann_concat_Mobile_conv.obs['leiden_0.5']

In [ ]:
tcell_ann_concat_Mobile_conv.obs['leiden_new_cluster_2']

In [ ]:
# Define a color for the new cluster (e.g., '#ff7f0e' for orange)
new_cluster_color = '#FFA500'

# Create the updated list of colors by appending the new color
updated_colors = ['#1f77b4','#2ca02c','#e377c2','#FFA500',]


In [ ]:
# Set the color map for leiden_new_cluster_2
tcell_ann_concat_Mobile_conv.uns['leiden_new_cluster_2_colors'] = updated_colors
#tcell_ann_concat_Mobile_conv.uns['leiden_new_cluster_colors'] = updated_colors

In [ ]:
#tcell_ann_concat_Mobile_conv[np.random.permutation(len(tcell_ann_concat_Mobile_conv))]

In [ ]:
#sc.settings.figdir = '/burg/iicd_new/users/ls3456/GVHD/Figures/Trajectory_figures/'
plt.figure(figsize=(15, 6))
# Now the plot will be saved in the specified directory
sc.pl.umap(tcell_ann_concat_Mobile_conv[np.random.permutation(len(tcell_ann_concat_Mobile_conv))],
    color=['leiden_new_cluster_2'],
    show=False,  # Do not show the plot yet,
    frameon=False,
    size=25,
    legend_loc= 'right margin',
    legend_fontsize = 20)

plt.gca().set_title('')

# Save the figure at 300 DPI
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Change_Resi_umap.png", dpi=300,bbox_inches='tight')
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Change_Resi_umap.svg", dpi=300,bbox_inches='tight')
plt.savefig("/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/leiden_Change_Resi_umap.pdf", dpi=300,bbox_inches='tight')

In [ ]:
tcell_ann_concat_Mobile_conv

In [ ]:
sc.tl.rank_genes_groups(tcell_ann_concat_Mobile_conv, groupby='leiden_new_cluster_2', use_raw = False, method = 'wilcoxon')

dedf = sc.get.rank_genes_groups_df(tcell_ann_concat_Mobile_conv, group=None)
dedf

dedf = dedf[dedf['scores']>=0]
dedf

dedf = dedf[dedf['pvals_adj']<=0.00000000000001]
dedf

gene_mask = tcell_ann_concat_Mobile_conv.var_names.isin(dedf['names'])
# Use the boolean mask to select the desired variables (genes)
tcell_ann_concat_Mobile_conv_deg = tcell_ann_concat_Mobile_conv[:, gene_mask]
tcell_ann_concat_Mobile_conv_deg

In [ ]:
#tcell_ann_concat_Mobile_conv_deg.write('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/tcell_ann_concat_Mobile_conv_deg.h5ad')

In [ ]:
sc.pl.umap(tcell_ann_concat_Mobile_conv, color=['SampleID_Grade' ])

In [ ]:
sc.tl.rank_genes_groups(tcell_ann_concat_Mobile_conv, groupby='leiden_new_cluster_2', use_raw=False, method='wilcoxon')
degs_df = sc.get.rank_genes_groups_df(tcell_ann_concat_Mobile_conv, group=None)

In [ ]:
#degs_df.to_csv('/burg/iicd/users/ls3456/GVHD/adata_file/cluster_deg.csv')

In [ ]:
#Resi.write('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/Resi.h5ad')

In [ ]:
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator  # Assuming you use statannotations for the p-values
from copy import deepcopy

In [ ]:
#!pip install statannotations

In [ ]:
from copy import deepcopy
from matplotlib import rcParams
from statannotations.Annotator import Annotator
import matplotlib as mpl
sc.settings.set_figure_params(dpi_save=300)
mpl.rcParams['axes.linewidth'] = 1.3 
mpl.rcParams['axes.grid'] = False  # Disable grid lines

# Assuming adata is already defined and contains the necessary data
obs_1 = 'SampleID_Grade'
obs_2 = 'leiden_new_cluster_2'
num = 1

for adata in [tcell_ann_concat_Mobile_conv]:
    n_categories = {x: len(adata.obs[x].cat.categories) for x in [obs_1, obs_2]}
    df = adata.obs[[obs_2, obs_1]].values

    obs2_clusters = adata.obs[obs_2].cat.categories.tolist()
    obs1_clusters = adata.obs[obs_1].cat.categories.tolist()

    obs1_to_obs2 = {k: np.zeros(len(obs2_clusters), dtype="i")
                    for k in obs1_clusters}
    obs2_to_obs1 = {k: np.zeros(len(obs1_clusters), dtype="i")
                    for k in obs2_clusters}
    for b, l in df:
        obs2_to_obs1[b][obs1_clusters.index(str(l))] += 1
        obs1_to_obs2[l][obs2_clusters.index(str(b))] += 1
    obs2_to_obs1_array = np.zeros((len(obs2_clusters), len(obs1_clusters)))
    obs1_to_obs2_array = np.zeros((len(obs1_clusters), len(obs2_clusters)))
    for i, k in enumerate(obs2_clusters):
        obs2_to_obs1_array[i, :] = deepcopy(obs2_to_obs1[k])
    for i, k in enumerate(obs1_clusters):
        obs1_to_obs2_array[i, :] = deepcopy(obs1_to_obs2[k])

    sums = np.sum(obs2_to_obs1_array, 0)
    for col in range(np.size(obs2_to_obs1_array, 1)):
        for row in range(np.size(obs2_to_obs1_array, 0)):
            obs2_to_obs1_array[row, col] = obs2_to_obs1_array[row, col] / sums[col]

    sums2 = np.sum(obs1_to_obs2_array, 0)
    for col in range(np.size(obs1_to_obs2_array, 1)):
        for row in range(np.size(obs1_to_obs2_array, 0)):
            obs1_to_obs2_array[row, col] = obs1_to_obs2_array[row, col] / sums2[col]

    df_obs2 = pd.DataFrame(obs2_to_obs1_array)
    df_obs2.index = obs2_clusters
    df_obs2.columns = obs1_clusters

    # Sort columns if necessary
    sorter = sorted(obs1_clusters)
    df_obs2 = df_obs2.reindex(columns=sorter)

    df_obs2.T.plot(kind="bar", stacked=True, color=adata.uns[obs_2 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    df_obs1 = pd.DataFrame(obs1_to_obs2_array)
    df_obs1.index = obs1_clusters
    df_obs1.columns = obs2_clusters
    df_obs1.T.plot(kind="bar", stacked=True, color=adata.uns[obs_1 + '_colors'])
    plt.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
    plt.show()

    print(num)
    num += 1
    p_list = []

    for i in adata.obs['leiden_new_cluster_2'].unique():
        substring = 'Severe'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_severe = df_obs2.loc[:, column_mask]
        severe = selected_columns_severe.loc[i].values[~np.isnan(selected_columns_severe.loc[i].values)]

        substring = 'Mild'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_mild = df_obs2.loc[:, column_mask]
        mild = selected_columns_mild.loc[i].values[~np.isnan(selected_columns_mild.loc[i].values)]

        substring = 'No'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_no = df_obs2.loc[:, column_mask]
        no = selected_columns_no.loc[i].values[~np.isnan(selected_columns_no.loc[i].values)]

        substring = 'Treated'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_treated = df_obs2.loc[:, column_mask]
        treated = selected_columns_treated.loc[i].values[~np.isnan(selected_columns_treated.loc[i].values)]


        substring = 'ND'
        column_mask = df_obs2.columns.str.contains(substring)
        selected_columns_nd = df_obs2.loc[:, column_mask]
        nd = selected_columns_nd.loc[i].values[~np.isnan(selected_columns_nd.loc[i].values)]

        
        df_severe = pd.DataFrame()
        df_severe['Value'] = severe
        df_severe['Grade'] = 'severe'

        df_mild = pd.DataFrame()
        df_mild['Value'] = mild
        df_mild['Grade'] = 'mild'

        df_no = pd.DataFrame()
        df_no['Value'] = no
        df_no['Grade'] = 'no'

        df_treated = pd.DataFrame()
        df_treated['Value'] = treated
        df_treated['Grade'] = 'no'

        df_ND = pd.DataFrame()
        df_ND['Value'] = nd
        df_ND['Grade'] = 'no'
        

        palette = {'no': '#09BB8C', 'mild': '#005A8F', 'severe': '#B85000'}

        result = pd.concat([df_no, df_treated, df_ND,df_mild, df_severe])
        rcParams.update({'font.size': 12})
        fig, ax = plt.subplots(figsize=(4, 2), dpi=300)
        sns.boxplot(x='Grade', y='Value', data=result, ax=ax, palette=palette, order=['no', 'mild', 'severe'], showfliers=False)
        sns.stripplot(x='Grade', y='Value', data=result, ax=ax, color='black', alpha=0.5, jitter=True, order=['no', 'mild', 'severe'])

        ax.spines[['right', 'top']].set_visible(False)
        ax.set_xlabel('')
        ax.set_ylabel('Cell type proportion')
        ax.set_title(i)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()

        pairs = [('no', 'mild'), ('no', 'severe'),('mild', 'severe')]
        annotator = Annotator(ax, pairs, data=result, x='Grade', y='Value')
        annotator.configure(test='Mann-Whitney', comparisons_correction="holm", correction_format="replace")
        
        annotator.apply_and_annotate()
        plt.tight_layout()

        # Save figure with 300 DPI
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_resi_{i}_boxplot_300dpi.png', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_resi_{i}_boxplot_300dpi.pdf', dpi=300)
        plt.savefig(f'/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/fig_resi_{i}_boxplot_300dpi.svg', dpi=300)
        plt.show()

In [ ]:
df_obs2

In [ ]:
 # Access the .obs 
from scipy.stats import fisher_exact
for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_0.5', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    normalized_pivot_table = normalized_pivot_table*100
    print(normalized_pivot_table)
    try:
        # Perform chi-square test
        chi2_stat, p_val, dof, ex = chi2_contingency(normalized_pivot_table)
    
        # Check if there are any zeroes in the expected frequencies
        if np.any(ex == 0):
            print("Skipping this chi-square test due to zero expected frequencies.")
        else:
            # Output the results if the test is valid
            print(f"Chi-square statistic: {chi2_stat}")
            print(f"P-value: {p_val}")
            print(f"Degrees of freedom: {dof}")
            print(f"Expected frequencies:\n{ex}")
    
    except ValueError as e:
        print(f"Skipping this test due to error: {e}")

import plotly.graph_objects as go
link_colors = [
    'rgba(31, 119, 180, 0.3)',  # Blue for 'Infiltrating activated cytotoxic T-cells'
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(44, 160, 44, 0.3)',   # Green for 'Resident CD8+ T-cells'
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(227, 119, 194, 0.3)', # Pink for 'Proliferating CD8+ T-cells'
    'rgba(227, 119, 194, 0.3)', # Pink
    'rgba(227, 119, 194, 0.3)'  # Pink
]

for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_0.5', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    print(normalized_pivot_table)
    
    df= pd.DataFrame()
    df_before =pd.DataFrame()
    for t_cell_states in tcell_ann_concat_Mobile_conv.obs['leiden_0.5'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_0.5'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_lp.obs['cc_aa_alignment_same_v_Grade'])]
        print(len(adata_migrate))
        
        adata_migrate.obs['leiden_0.5'] = pd.Categorical(adata_migrate.obs['leiden_0.5'], categories=['Infiltrating activated cytotoxic T-cells','Resident CD8+ T-cells', 'Proliferating CD8+ T-cells'])
        df[t_cell_states] = adata_migrate.obs['leiden_0.5'].value_counts(normalize=True) * normalized_pivot_table['Lamina Propria Cells'][t_cell_states]
        print(adata_migrate.obs['leiden_0.5'].unique())

    print(df)
    df.fillna(0, inplace=True)

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = ["LP_Activated", "LP_Resident", "LP_Proliferating", "IE_Activated", "IE_Resident", "IE_Proliferating"],
          color = ['#1f77b4','#2ca02c','#e377c2','#1f77b4','#2ca02c','#e377c2']
        ),
        link = dict(
          source = [0, 0, 0, 1, 1, 1,2,2,2], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [3, 4, 5, 3, 4, 5,3,4,5],
          value = [df['Infiltrating activated cytotoxic T-cells'][0], df['Infiltrating activated cytotoxic T-cells'][1] , df['Infiltrating activated cytotoxic T-cells'][2] ,df['Resident CD8+ T-cells'][0] , df['Resident CD8+ T-cells'][1], df['Resident CD8+ T-cells'][2] ,df['Proliferating CD8+ T-cells'][0] , df['Proliferating CD8+ T-cells'][1], df['Proliferating CD8+ T-cells'][2] ],
            color = link_colors,
      ))])
    
    fig.update_layout(title_text=i, font_size=10)
    fig.show()
    

In [ ]:
tcell_ann_concat_Mobile_conv.obs['leiden_new_cluster_2']

In [ ]:
import plotly.graph_objects as go
from anndata import AnnData
### Remove them

link_colors = [
    'rgba(31, 119, 180, 0.3)',  # Blue for 'Infiltrating activated cytotoxic T-cells'
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(44, 160, 44, 0.3)',   # Green for 'Resident CD8+ T-cells'
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(227, 119, 194, 0.3)', # Pink for 'Proliferating CD8+ T-cells'
    'rgba(227, 119, 194, 0.3)', # Pink
    'rgba(227, 119, 194, 0.3)',  # Pink
    'rgba(227, 119, 194, 0.3)'  # Pink
]

for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_new_cluster_2', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    print(normalized_pivot_table)
    
    df= pd.DataFrame()
    df_before =pd.DataFrame()
    total_migrate_count = 0
    total_lp_count = 0
    accumulated_adata_migrate = []
    accumulated_adata_lp_count = []
    for t_cell_states in tcell_ann_concat_Mobile_conv.obs['leiden_new_cluster_2'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_new_cluster_2'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_lp.obs['cc_aa_alignment_same_v_Grade'])]
        adata_lp_count= adata_lp[adata_lp.obs['cc_aa_alignment_same_v_Grade'].isin(adata_migrate.obs['cc_aa_alignment_same_v_Grade'])]
        adata_ie_count= adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_migrate.obs['cc_aa_alignment_same_v_Grade'])]
        
        total_migrate_count += len(adata_migrate)
        total_lp_count += len(adata_lp_count)
        
        #print("adata_migrate count:",len(adata_migrate))
        #print("adata_lp count:",len(adata_lp_count))
        #print("adata_ie count:",len(adata_ie_count))
        
        # Concatenate the new data to the accumulated data
        accumulated_adata_migrate = accumulated_adata_migrate+list(adata_migrate.obs.index)
        accumulated_adata_lp_count = accumulated_adata_lp_count+list(adata_lp_count.obs.index)
        
        adata_migrate.obs['leiden_new_cluster_2'] = pd.Categorical(adata_migrate.obs['leiden_new_cluster_2'], categories=['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells','CD8+ Transitioning Resident T cells'])
        df[t_cell_states] = adata_migrate.obs['leiden_new_cluster_2'].value_counts(normalize=True) * normalized_pivot_table['Lamina Propria Cells'][t_cell_states]
        #print(adata_migrate.obs['leiden_new_cluster_2'].unique())

    
    df.fillna(0, inplace=True)

    if (df == 0).any().any():
        print("The DataFrame contains 0.")
    else:
        print("The DataFrame does not contain any 0.")
    
    print('df')
    print(df)
    print('sum of counts')
    print("Total adata_migrate count:", total_migrate_count)
    print("Total adata_lp_count count:", total_lp_count)
    print('sum of unique cells')
    print("Total adata_migrate count:", len(list(set(accumulated_adata_migrate))))
    print("Total adata_lp_count count:", len(list(set(accumulated_adata_lp_count))))


    

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = [],
          color = ['#1f77b4','#FFA500','#1b7837','#e377c2','#1f77b4','#FFA500','#1b7837','#e377c2']
        ),
        link = dict(
          source = [0, 0, 0,0, 1, 1, 1,1,2,2,2,2,3,3,3,3], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [4, 5, 6, 7,4, 5, 6, 7,4, 5, 6, 7,4, 5, 6, 7],
          value = [df['CD8+ Effector T cells']['CD8+ Effector T cells'], df['CD8+ Effector T cells']['CD8+ Transitioning Resident T cells'] , df['CD8+ Effector T cells']['CD8+ Tissue Resident Memory T cells'] , df['CD8+ Effector T cells']['Proliferating CD8+ T cells'], df['CD8+ Transitioning Resident T cells']['CD8+ Effector T cells'] , df['CD8+ Transitioning Resident T cells']['CD8+ Transitioning Resident T cells'], df['CD8+ Transitioning Resident T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Transitioning Resident T cells']['Proliferating CD8+ T cells'], df['CD8+ Tissue Resident Memory T cells']['CD8+ Effector T cells'] , df['CD8+ Tissue Resident Memory T cells']['CD8+ Transitioning Resident T cells'], df['CD8+ Tissue Resident Memory T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Tissue Resident Memory T cells']['Proliferating CD8+ T cells'],df['Proliferating CD8+ T cells']['CD8+ Effector T cells'], df['Proliferating CD8+ T cells']['CD8+ Transitioning Resident T cells'] , df['Proliferating CD8+ T cells']['CD8+ Tissue Resident Memory T cells'],df['Proliferating CD8+ T cells']['Proliferating CD8+ T cells']],
            color = link_colors,
      ))])
    
    fig.update_layout(title_text=i,
                  font=dict(family="Arial", size=18, color="black"))
    fig.show()
        # Save as PNG
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram_nolabel.png", scale=4)
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram_nolabel.svg", scale=4)
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram_nolabel.pdf", scale=4)

In [ ]:
import plotly.graph_objects as go
from anndata import AnnData
### Remove them

link_colors = [
    'rgba(31, 119, 180, 0.3)',  # Blue for 'Infiltrating activated cytotoxic T-cells'
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(31, 119, 180, 0.3)',  # Blue
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(255, 165, 0, 0.3)',
    'rgba(44, 160, 44, 0.3)',   # Green for 'Resident CD8+ T-cells'
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(44, 160, 44, 0.3)',   # Green
    'rgba(227, 119, 194, 0.3)', # Pink for 'Proliferating CD8+ T-cells'
    'rgba(227, 119, 194, 0.3)', # Pink
    'rgba(227, 119, 194, 0.3)',  # Pink
    'rgba(227, 119, 194, 0.3)'  # Pink
]

for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_new_cluster_2', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    print(normalized_pivot_table)
    
    df= pd.DataFrame()
    df_before =pd.DataFrame()
    total_migrate_count = 0
    total_lp_count = 0
    accumulated_adata_migrate = []
    accumulated_adata_lp_count = []
    for t_cell_states in tcell_ann_concat_Mobile_conv.obs['leiden_new_cluster_2'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_new_cluster_2'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v_Grade'].isin(adata_lp.obs['cc_aa_alignment_same_v_Grade'])]
        adata_lp_count= adata_lp[adata_lp.obs['cc_aa_alignment_same_v_Grade'].isin(adata_migrate.obs['cc_aa_alignment_same_v_Grade'])]
        total_migrate_count += len(adata_migrate)
        total_lp_count += len(adata_lp_count)
        
        print(len(adata_migrate))
        print(len(adata_lp_count))
        # Concatenate the new data to the accumulated data
        accumulated_adata_migrate = accumulated_adata_migrate+list(adata_migrate.obs.index)
        accumulated_adata_lp_count = accumulated_adata_lp_count+list(adata_lp_count.obs.index)
        
        adata_migrate.obs['leiden_new_cluster_2'] = pd.Categorical(adata_migrate.obs['leiden_new_cluster_2'], categories=['CD8+ Tissue Resident Memory T cells','CD8+ Effector T cells','Proliferating CD8+ T cells','CD8+ Transitioning Resident T cells'])
        df[t_cell_states] = adata_migrate.obs['leiden_new_cluster_2'].value_counts(normalize=True) * normalized_pivot_table['Lamina Propria Cells'][t_cell_states]
        df_count[t_cell_states] = adata_migrate.obs['leiden_new_cluster_2'].value_counts(normalize=True) * normalized_pivot_table['Lamina Propria Cells'][t_cell_states]
        #print(adata_migrate.obs['leiden_new_cluster_2'].unique())

    
    df.fillna(0, inplace=True)

    if (df == 0).any().any():
        print("The DataFrame contains 0.")
    else:
        print("The DataFrame does not contain any 0.")
    
    print('df')
    print(df)
    print('sum of counts')
    print("Total adata_migrate count:", total_migrate_count)
    print("Total adata_lp_count count:", total_lp_count)
    print('sum of unique cells')
    print("Total adata_migrate count:", len(list(set(accumulated_adata_migrate))))
    print("Total adata_lp_count count:", len(list(set(accumulated_adata_lp_count))))


    

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = ["CD8+ Effector", "CD8+ Transitioning Resident","CD8+ Tissue Resident Memory", "CD8+ Proliferating ","CD8+ Effector", "CD8+ Transitioning Resident","CD8+ Tissue Resident Memory", "CD8+ Proliferating "],
          color = ['#1f77b4','#FFA500','#1b7837','#e377c2','#1f77b4','#FFA500','#1b7837','#e377c2']
        ),
        link = dict(
          source = [0, 0, 0,0, 1, 1, 1,1,2,2,2,2,3,3,3,3], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [4, 5, 6, 7,4, 5, 6, 7,4, 5, 6, 7,4, 5, 6, 7],
          value = [df['CD8+ Effector T cells']['CD8+ Effector T cells'], df['CD8+ Effector T cells']['CD8+ Transitioning Resident T cells'] , df['CD8+ Effector T cells']['CD8+ Tissue Resident Memory T cells'] , df['CD8+ Effector T cells']['Proliferating CD8+ T cells'], df['CD8+ Transitioning Resident T cells']['CD8+ Effector T cells'] , df['CD8+ Transitioning Resident T cells']['CD8+ Transitioning Resident T cells'], df['CD8+ Transitioning Resident T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Transitioning Resident T cells']['Proliferating CD8+ T cells'], df['CD8+ Tissue Resident Memory T cells']['CD8+ Effector T cells'] , df['CD8+ Tissue Resident Memory T cells']['CD8+ Transitioning Resident T cells'], df['CD8+ Tissue Resident Memory T cells']['CD8+ Tissue Resident Memory T cells'] ,df['CD8+ Tissue Resident Memory T cells']['Proliferating CD8+ T cells'],df['Proliferating CD8+ T cells']['CD8+ Effector T cells'], df['Proliferating CD8+ T cells']['CD8+ Transitioning Resident T cells'] , df['Proliferating CD8+ T cells']['CD8+ Tissue Resident Memory T cells'],df['Proliferating CD8+ T cells']['Proliferating CD8+ T cells']],
            color = link_colors,
      ))])
    
    fig.update_layout(title_text=i,
                  font=dict(family="Arial", size=18, color="black"))
    fig.show()
        # Save as PNG
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.png", scale=4)
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.svg", scale=4)
    fig.write_image(f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.pdf", scale=4)

In [ ]:
# Update your figure creation and layout code
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["CD8+ Effector", "CD8+ Transitioning Resident","CD8+ Tissue Resident Memory", "CD8+ Proliferating ","CD8+ Effector", "CD8+ Transitioning Resident","CD8+ Tissue Resident Memory", "CD8+ Proliferating "],
      color = ['#1f77b4','#FFA500','#1b7837','#e377c2','#1f77b4','#FFA500','#1b7837','#e377c2']
    ),
    link = dict(
      source = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
      target = [4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7],
      value = [df['CD8+ Effector T cells']['CD8+ Effector T cells'], 
               df['CD8+ Effector T cells']['CD8+ Transitioning Resident T cells'],
               df['CD8+ Effector T cells']['CD8+ Tissue Resident Memory T cells'],
               df['CD8+ Effector T cells']['Proliferating CD8+ T cells'],
               df['CD8+ Transitioning Resident T cells']['CD8+ Effector T cells'],
               df['CD8+ Transitioning Resident T cells']['CD8+ Transitioning Resident T cells'],
               df['CD8+ Transitioning Resident T cells']['CD8+ Tissue Resident Memory T cells'],
               df['CD8+ Transitioning Resident T cells']['Proliferating CD8+ T cells'],
               df['CD8+ Tissue Resident Memory T cells']['CD8+ Effector T cells'],
               df['CD8+ Tissue Resident Memory T cells']['CD8+ Transitioning Resident T cells'],
               df['CD8+ Tissue Resident Memory T cells']['CD8+ Tissue Resident Memory T cells'],
               df['CD8+ Tissue Resident Memory T cells']['Proliferating CD8+ T cells'],
               df['Proliferating CD8+ T cells']['CD8+ Effector T cells'],
               df['Proliferating CD8+ T cells']['CD8+ Transitioning Resident T cells'],
               df['Proliferating CD8+ T cells']['CD8+ Tissue Resident Memory T cells'],
               df['Proliferating CD8+ T cells']['Proliferating CD8+ T cells']
              ],
      color = link_colors
    )
)])

# Update layout to make the background transparent
fig.update_layout(
    title_text=i,
    font=dict(family="Arial", size=18, color="black"),
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
    plot_bgcolor='rgba(0,0,0,0)'    # Transparent plot area
)

# Update the save options to include transparent background
fig.write_image(
    f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.png", 
    scale=4
)

fig.write_image(
    f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.svg", 
    scale=4
)

fig.write_image(
    f"/Users/lingting/Documents/GVHD_project/Paper_ready_pipeline/Figures/Trajectory_figures/{i}_sankey_diagram.pdf", 
    scale=4
)

In [ ]:
 # Access the .obs 
from scipy.stats import fisher_exact
for i in tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_conv[tcell_ann_concat_Mobile_conv.obs['GVHD_Grade_1'] == i]
    # Create a pivot table with 'PatientID' as rows, 'Treatment' as columns, and the mean of 'GeneExpression'
    obs_df = tcr_dataset.obs 
    pivot_table = pd.pivot_table(obs_df, values='cc_aa_alignment_same_v_Grade', index='leiden_new_cluster_2', columns='LPorIE', aggfunc='count')
    normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)
    normalized_pivot_table = normalized_pivot_table*100
    print(normalized_pivot_table)
    try:
        # Perform chi-square test
        chi2_stat, p_val, dof, ex = chi2_contingency(normalized_pivot_table)
    
        # Check if there are any zeroes in the expected frequencies
        if np.any(ex == 0):
            print("Skipping this chi-square test due to zero expected frequencies.")
        else:
            # Output the results if the test is valid
            print(f"Chi-square statistic: {chi2_stat}")
            print(f"P-value: {p_val}")
            print(f"Degrees of freedom: {dof}")
            print(f"Expected frequencies:\n{ex}")
    
    except ValueError as e:
        print(f"Skipping this test due to error: {e}")

In [ ]:
!pip install cellrank

In [ ]:
!pip install -U scvelo


In [ ]:
import numpy as np

import cellrank as cr
import scanpy as sc
import scvelo as scv

scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2


In [ ]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
# Recompute PCA with more components (e.g., 30 components)
scv.pp.pca(tcell_ann_concat_Mobile, n_comps=30)

# Verify that PCA was computed with the correct number of dimensions
print(tcell_ann_concat_Mobile.obsm['X_pca'].shape)


In [ ]:
scv.pp.neighbors(tcell_ann_concat_Mobile, n_pcs=30, n_neighbors=30)

In [ ]:
scv.pp.filter_and_normalize(tcell_ann_concat_Mobile, min_counts=30, min_counts_u=30)

In [ ]:
scv.pp.moments(tcell_ann_concat_Mobile, n_pcs=None, n_neighbors=None)

In [ ]:
import scvelo as scv

# Assuming you've already loaded and preprocessed your data (adata)
# Recover the transcriptional dynamics
#scv.tl.recover_dynamics(tcell_ann_concat_Mobile)

# Compute RNA velocity using the dynamical model
scv.tl.velocity(tcell_ann_concat_Mobile, mode='deterministic')

# Compute the velocity graph
scv.tl.velocity_graph(tcell_ann_concat_Mobile)

# Visualize the velocity on a UMAP or PCA embedding
scv.pl.velocity_embedding_stream(tcell_ann_concat_Mobile, basis='umap')


In [ ]:
scv.tl.recover_dynamics(tcell_ann_concat_Mobile)  # infers latent time and splicing kinetics
scv.tl.velocity(tcell_ann_concat_Mobile, mode='dynamical')
scv.tl.velocity_graph(tcell_ann_concat_Mobile)

In [ ]:
tcell_ann_concat_Mobile.obs['leiden_0.5'].unique()

In [ ]:
tcell_ann_concat_Mobile_unconv = tcell_ann_concat_Mobile[tcell_ann_concat_Mobile.obs['leiden_0.5'].isin(['Homeostatic unconventional CD8+ T cells','Activated Unconventional CD8+ T-cells'] )]

In [ ]:
for i in tcell_ann_concat_Mobile_unconv.obs['GVHD_Grade_1'].unique():
    print(i)
    tcr_dataset = tcell_ann_concat_Mobile_unconv[tcell_ann_concat_Mobile_unconv.obs['GVHD_Grade_1'] == i]
    df= pd.DataFrame()
    for t_cell_states in tcell_ann_concat_Mobile_unconv.obs['leiden_0.5'].unique():
        print(t_cell_states)
        adata = tcr_dataset[tcr_dataset.obs['leiden_0.5'] == t_cell_states]
        adata_lp = adata[adata.obs['LPorIE'] == 'Lamina Propria Cells']
        adata_ie = tcr_dataset[tcr_dataset.obs['LPorIE'] == 'Intraepithelial Cells']
        adata_migrate = adata_ie[adata_ie.obs['cc_aa_alignment_same_v'].isin(adata_lp.obs['cc_aa_alignment_same_v'])]
        print(len(adata_migrate))
        adata_migrate.obs['leiden_0.5'] = pd.Categorical(adata_migrate.obs['leiden_0.5'], categories=['Activated Unconventional CD8+ T-cells', 'Homeostatic unconventional CD8+ T cells'])
        df[t_cell_states] = adata_migrate.obs['leiden_0.5'].value_counts(normalize=True)
        print(adata_migrate.obs['leiden_0.5'].unique())

    print(df)
    df.fillna(0, inplace=True)

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = ["LP_Activated", "LP_Resident", "IE_Activated", "IE_Resident"],
          color = ['#9467bd', '#2ca02c','#9467bd', '#2ca02c']
        ),
        link = dict(
          source = [0, 0, 1, 1], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [2, 3, 2,3],
          value = [df['Activated Unconventional CD8+ T-cells'][0], df['Activated Unconventional CD8+ T-cells'][1] ,df['Homeostatic unconventional CD8+ T cells'][0] , df['Homeostatic unconventional CD8+ T cells'][1]]
      ))])
    
    fig.update_layout(title_text=i, font_size=10)
    fig.show()

In [ ]:
df['Infiltrating activated cytotoxic T-cells'][0]

In [ ]:
df

In [ ]:
adata_migrate

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=80)

In [ ]:
epithelial = anndata.read_h5ad('/Users/lingtingshi/Documents/GVHD_project/epithelial.h5ad')

In [ ]:
for i in epithelial.obs['GVHD_Grade_1'].unique():
    epi_sub = epithelial[epithelial.obs['GVHD_Grade_1'] == i]
    print(i)
    print(epi_sub)

In [ ]:
epi_sub = epithelial[epithelial.obs['GVHD_Grade_1'].isin(['Mild','Severe'])]
sc.tl.rank_genes_groups(epi_sub, groupby="GVHD_Grade_1", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(epi_sub, group=None)
dedf.to_csv('/Users/lingtingshi/Documents/GVHD_project/Mild_Severe_w_notraw.csv')


In [ ]:
epi_sub = epithelial[epithelial.obs['GVHD_Grade_1'].isin(['Mild','No'])]
sc.tl.rank_genes_groups(epi_sub, groupby="GVHD_Grade_1", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(epi_sub, group=None)
dedf.to_csv('/Users/lingtingshi/Documents/GVHD_project/Mild_No_w_notraw.csv')

In [ ]:
epi_sub = epithelial[epithelial.obs['GVHD_Grade_1'].isin(['Severe','No'])]
sc.tl.rank_genes_groups(epi_sub, groupby="GVHD_Grade_1", use_raw=False, method='wilcoxon')
dedf = sc.get.rank_genes_groups_df(epi_sub, group=None)
dedf.to_csv('/Users/lingtingshi/Documents/GVHD_project/Severe_No_w_notraw.csv')